In [3]:
#!pip install geopandas

In [4]:
import pandas as pd
import urllib.request
import json

import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, Point

# fires

In [5]:
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=8d76ac6b-5ae8-4428-82a4-043130d17b02&limit=32000'
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

dfFire = pd.concat(dfs.values(), ignore_index = True)

# isolate 'type', 'desc', 'location', 'date', 'zone_police', 'zone_fire', 'lat', 'lng'
dfFire = dfFire.loc[:, ['alarm_time', 'latitude', 'longitude']]

# rename columns
dfFire.columns = ['date', 'lat', 'lng'] # 'zone' is 'police_zone'

# drop nulls
dfFire = dfFire.dropna()
# just a few nulls, ignoring nulls in 'zone_police' and 'lat' and 'lng'

# change 'date' to columns ['month', 'year']
dfFire['date'] = pd.to_datetime(dfFire['date']).apply(lambda x: x.strftime('%Y-%m'))

# crimes

In [6]:
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    apiURL = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=044f2016-1dfd-4ab0-bc1e-065da05fca2e&limit=320000'
    apiURL += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(apiURL)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

dfCrime = pd.concat(dfs.values(), ignore_index = True)

# isolate 'date', 'location', 'zone', 'desc', 'lng', 'lat'
dfCrime = dfCrime.loc[:, ['INCIDENTTIME', 'X', 'Y']]

# rename columns
dfCrime.columns = ['date', 'lng', 'lat']

# drop nulls
dfCrime = dfCrime.dropna()
# can't do anything without a desc and there were only four more instances of datetime being null so might as well drop
# also need non-null 'lat' and 'lng' to reassign 'zone' with knn preds (future step)

# change 'date' to columns ['month', 'year']
dfCrime['date'] = pd.to_datetime(dfCrime['date']).apply(lambda x: x.strftime('%Y-%m'))

# rearrange 'lng' and 'lat' columns - purely personal aesthetic choice
# make the new columns (in the right order)
dfCrime['placeholder_lat'] = dfCrime['lat']
dfCrime['placeholder_lng'] = dfCrime['lng']
# drop the old columns
dfCrime = dfCrime.drop(columns = ['lng', 'lat'])
# rename the new columns
dfCrime = dfCrime.rename(columns = {'placeholder_lat': 'lat', 'placeholder_lng': 'lng'})

# ensure 'lat' and 'lng' are float type
dfCrime['lat'], dfCrime['lng'] = dfCrime['lat'].astype(float), dfCrime['lng'].astype(float)

# drop nulls one final time (I guess because we need to?)
dfCrime = dfCrime.dropna()

# parcels

In [7]:
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=adf1fd38-c374-4c4e-9094-5e53bd12419f&limit=32000'
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

dfParcels = pd.concat(dfs.values(), ignore_index = True)

# isolate 'parcelID', 'lat', 'lng'
dfParcels = dfParcels.loc[:, ['PIN', 'Latitude', 'Longitude']]

# rename columns
dfParcels.columns = ['parcelID', 'lat', 'lng']

# drop nulls - string 'nan' in this case
dfParcels = dfParcels.drop(index = [583953, 583954])

# sales

In [8]:
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=5bbe6c55-bce6-4edb-9d04-68edeb6bf7b1&limit=32000'
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

df1 = pd.concat(dfs.values(), ignore_index = True)

# isolate 'parcelID', 'date', 'salePrice', where 'PROPERTYCITY' == 'PITTSBURGH'
df1 = df1.loc[df1['PROPERTYCITY'] == 'PITTSBURGH', ['PARID', 'SALEDATE', 'PRICE']]

# rename columns
df1.columns = ['parcelID', 'date', 'salePrice']

# drop nulls
df1 = df1.dropna(subset = ['salePrice'])

# change 'date' to columns ['month', 'year']
df1['date'] = pd.to_datetime(df1['date']).apply(lambda x: x.strftime('%Y-%m'))

# ------------------------------------------------------------------------------------------------------------------------ #

# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=f2b8d575-e256-4718-94ad-1e12239ddb92&limit=32000'
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

df2 = pd.concat(dfs.values(), ignore_index = True)

# isolate 'parcelID', 'date', 'salePrice', where 'PROPERTYCITY' == 'PITTSBURGH'
df2 = df2.loc[df2['PROPERTYCITY'] == 'PITTSBURGH', ['PARID', 'SALEDATE', 'SALEPRICE']]

# rename columns
df2.columns = ['parcelID', 'date', 'salePrice']

# drop nulls
df2 = df2.dropna(subset = ['salePrice'])

# change 'date' to columns ['month', 'year']
df2['date'] = pd.to_datetime(df2['date']).apply(lambda x: x.strftime('%Y-%m'))

# ------------------------------------------------------------------------------------------------------------------------ #

dfSales = pd.concat([df1, df2], axis = 0).drop_duplicates(subset = ['parcelID', 'date', 'salePrice'])

# ------------------------------------------------------------------------------------------------------------------------ #

dfSales = dfSales.merge(right = dfParcels, on = 'parcelID')

# code violations

In [11]:
# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=70c06278-92c5-4040-ab28-17671866f81c&limit=32000'
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

dfViols = pd.concat(dfs.values(), ignore_index = True)

# isolate 'parcelID', 'date', 'violation'
dfViols = dfViols.loc[:, ['parcel_id', 'investigation_date', 'violation_code_section']]

# rename columns
dfViols.columns = ['parcelID', 'date', 'violation']

# drop nulls
dfViols = dfViols.dropna()

# change 'date' to columns ['month', 'year']
dfViols['date'] = pd.to_datetime(dfViols['date']).apply(lambda x: x.strftime('%Y-%m'))

# _____________________________________________________________________________________________________________________________________________________
# extra df of historical code violations

# dictionary to store temporary dfs
dfs = {}

# download the data via the provided API
offset = 0
index = 0
while True:
    url = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id=4e5374be-1a88-47f7-afee-6a79317019b4&limit=32000'  
    url += '&offset=' + str(offset)
    fileobj = urllib.request.urlopen(url)
    data = pd.DataFrame(json.loads(fileobj.read())['result']['records'])
    dfs['df' + str(index)] = data
    offset += 32000
    index += 1
    if len(data) == 0:
        break

dfViols_historical = pd.concat(dfs.values(), ignore_index = True)

# isolate 'date', violation', 'parcelID', 'lng', 'lat'
dfViols_historical = dfViols_historical.loc[:, ['INSPECTION_DATE', 'VIOLATION', 'PARCEL', 'X', 'Y']]

# rename columns
dfViols_historical.columns = ['date', 'violation', 'parcelID', 'lng', 'lat']

# drop nulls
dfViols_historical = dfViols_historical.dropna()

# change 'date' to columns ['month', 'year']
dfViols_historical['date'] = pd.to_datetime(dfViols_historical['date']).apply(lambda x: x.strftime('%Y-%m'))

# filtering

In [15]:
geojson = gpd.read_file("C:\\Users\\Taavi\\Downloads\\Pittsburgh_Boundary.geojson")

boundary = geojson.geometry.iloc[2]
# this is the perimeter of the city

polygons = [geom for geom in boundary.geoms]
    
def filterData(data, polygons):
    filteredData = []
    for index, row in data.iterrows():
        dataPoint = Point(row['lng'], row['lat'])
        for polygon in polygons:
            if dataPoint.within(polygon):
                filteredData.append(row)
                break
    return pd.DataFrame(filteredData)

fire = filterData(dfFire, polygons)
crime = filterData(dfCrime, polygons)
sales = filterData(dfSales, polygons)
parcels = filterData(dfParcels, polygons)
dfViols = dfViols.merge(right = parcels, on = 'parcelID', how = 'inner')
dfViols_historical = dfViols_historical.drop(columns = ['lng', 'lat']).merge(right = parcels, on = 'parcelID', how = 'inner')
viols = pd.concat([dfViols, dfViols_historical], axis = 0)

# reset indexes

In [16]:
fire = fire.reset_index().drop(columns = 'index')
crime = crime.reset_index().drop(columns = 'index')
parcels = parcels.reset_index().drop(columns = 'index')
sales = sales.reset_index().drop(columns = 'index')
viols = viols.reset_index().drop(columns = 'index')

# export csvs

In [17]:
fire.to_csv('C:\\Users\\Taavi\\Downloads\\rawFire.csv', index = False)

In [18]:
crime.to_csv('C:\\Users\\Taavi\\Downloads\\rawCrime.csv', index = False)

In [19]:
parcels.to_csv('C:\\Users\\Taavi\\Downloads\\rawParcels.csv', index = False)

In [20]:
sales.to_csv('C:\\Users\\Taavi\\Downloads\\rawSales.csv', index = False)

In [21]:
viols.to_csv('C:\\Users\\Taavi\\Downloads\\rawViols.csv', index = False)